In [1]:
import chess
import torch

from chess_app.environment import ChessInterface
from chess_app.chess_exceptions import NotEnoughMovesExceptions
from re_algoritms.agents import RandomAgent, QAgent
from chess_app.dataset.loader import loaderChessPos, tanstsovVecLoader
import chess.svg
from tqdm import tqdm


In [2]:

def is_checkmate(board, state):
    board.set_fen(state)
    return board.is_checkmate()


In [3]:
from re_algoritms.dqn import GrandMasterNet
from re_algoritms.agents import DQNAgent

board = chess.Board()

lr = 1e-2
gamma = 0.7
n_epochs = 100

policy_net = GrandMasterNet(
    board_vec_dim=68,
    moves_vec_dim=5
)
player = DQNAgent(
    lr=lr, gamma=gamma, model=policy_net
)
loader = tanstsovVecLoader(r'chess_app\dataset\result.json', r'chess_app\dataset\settings.json')
loader.createDf()
frac = 0.4
dataset = loader.getAllData(frac=frac)
fen_strings = dataset.iloc[:, 0]
dataset = dataset.iloc[:, 1:]

In [4]:
dataset

,white_king_castle,white_queen_castle,black_king_castle,black_queen_castle,a8,b8,c8,d8,e8,f8,...,g2,h2,a1,b1,c1,d1,e1,f1,g1,h1
4142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
661,0,0,0,0,7,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7453,0,0,0,0,0,9,0,0,7,0,...,5,5,0,0,0,3,0,0,1,0
1039,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7730,0,0,0,0,9,0,10,0,9,7,...,5,5,3,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6900,0,0,0,0,0,9,0,0,9,0,...,0,5,0,0,0,0,0,0,1,0
3121,0,0,0,0,9,0,0,0,0,7,...,0,5,0,1,0,0,3,0,0,0
8109,0,0,0,0,0,0,0,0,0,9,...,1,0,0,0,0,0,0,0,0,0
2419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
for array, fen_string in zip(dataset.to_numpy(), fen_strings):
    # print(_)
    print(array, fen_string)
    break

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0
  0  0  0 11  0  0  0  0  0  1 11  7  0  0  0  0  4  4  0 11  0  0  0  0
  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 8/5p2/7p/5Kpk/4BB1p/7r/8/8 w - - 0 1


In [6]:
import wandb

wandb.init(
    project='chess-rl',
    config={
        'epochs': n_epochs,
        'lr': lr,
        'gamma': gamma,
        'frac': frac,
        'dataset_len': len(fen_strings)
    }
)

wandb.run.name = f'BoardNet: {3}, MovesNet: {2}, OutputNet: {2}, activation: leaky-relu, lr={lr}, gamma={gamma}'

wandb.define_metric("loss", summary="min")
wandb.define_metric("loss_epoch", summary="min")

wandb.define_metric("win_rate", summary="max")
wandb.define_metric("win_rate_epoch", summary="max")

wandb.define_metric("lose_rate", summary="min")
wandb.define_metric("lose_rate_epoch", summary="min")

wandb.define_metric('stalemate_rate', summary="min")
wandb.define_metric('stalemate_rate_epoch', summary="min")

wandb.watch(player.policy_net, log='all', log_freq=10)

wandb: Currently logged in as: crazy_historian (ai_community). Use `wandb login --relogin` to force relogin


[]

In [7]:
def check_gradients(model):
    for params in model.parameters():
        if params.grad is not None:
            print(f'Mean: {torch.mean(params.grad.data.view(-1).detach())}')
            print(f'STD: { torch.std(params.grad.data.view(-1).detach())}')
            break

In [8]:
bc_win = 0
wh_win = 0
stockfish = ChessInterface(verbose=True, engine_path=r"./chess_app/src/stockfish_15_x64_avx2.exe")
metrics_l = []
metrics_w = []
losses = []


loss = None

player.reset_actions()
player.policy_net.train()
for epoch in range(n_epochs):
    print(f'\nEpoch num: {epoch}\n')
    count_w_l = [0, 0, 0]
    for num, (array, fen_string) in enumerate(zip(dataset.to_numpy(), fen_strings)):
        # print(f'Party №{num + 857}')
        # print(fen_string)
        # init necessary for game
        new_state = fen_string
        new_array = array

        black_win = False
        white_win = False
        stalemate = False
        too_much_moves = False

        stockfish.set_board_fen(new_state)
        all_rewards = list()

        for _ in range(4):
            # player make move
            stockfish.set_board_fen(new_state)
            try:
                player_move, move_num = player.return_move(
                    new_array,
                    stockfish.get_top_steps() # todo
                )

                stockfish.player_move(player_move)
                new_state = stockfish.get_board_fen()

                all_rewards.append(0)
                # print(f'\tPlayer move: {player_move}, num of move: {move_num}')
                if is_checkmate(board, new_state):
                    white_win = True
                    break
            except NotEnoughMovesExceptions:
                # print('NotEnoughMovesExceptions')
                if len(all_rewards) < 1:
                    all_rewards.append(-1)
                else:
                    all_rewards[-1] = 1
                black_win = True
                break

            # environment make move
            try:
                machine_move = stockfish.env_move()
                new_state = stockfish.get_board_fen()
                # print(f'\tMachine move: {machine_move}')
                new_array = loader.extractPos(new_state, -1, -1)
                new_array = list(new_array.values())[3:]

                if is_checkmate(board, new_state):
                    black_win = True
                    break

            except ValueError as err:
                # print('Произошел пат!!!!!!!!!!!!!')
                stalemate = True
                count_w_l[2] += 1
                break

        else:
            # print('> 4 ходов')
            count_w_l[2] += 1
            too_much_moves = True


        if black_win is True:
            reward = -1
            count_w_l[1] += 1
        elif stalemate is True:
            reward = - 0.1
        elif too_much_moves is True:
            reward = 0.2
        else:
            reward = 1
            count_w_l[0] += 1

        all_rewards[-1] = reward
        loss = player.update_policy(all_rewards)
        if num % 20 == 0:
            print(f'Loss value: {loss}, num of party: {num}')
            print(f'Win rate: {count_w_l[0]}')
            print(f'Lose rate: {count_w_l[1]}')
            print(f'Stalemate rate: {count_w_l[2]}')

            wandb.log({
                'loss': loss,
                'win_rate': count_w_l[0]/ len(fen_strings),
                'lose_rate': count_w_l[1]/ len(fen_strings),
                'stalemate_rate': count_w_l[2]/ len(fen_strings)
             })

            losses.append(loss)
    wandb.log({
        'loss_epoch': loss,
        'win_rate_epoch': count_w_l[0]/ len(fen_strings),
        'lose_rate_epoch': count_w_l[1]/ len(fen_strings),
        'stalemate_rate_epoch': count_w_l[2]/ len(fen_strings)
    })

    metrics_l.append(count_w_l[1]/len(fen_strings))
    metrics_w.append(count_w_l[0]/len(fen_strings))
    print(f'Winrate {count_w_l[0]} \t Loserate {count_w_l[1]}')
    torch.save(player.policy_net.state_dict(), 'weight.pth')



Epoch num: 0

Loss value: -5.659895896911621, num of party: 0
Win rate: 0
Lose rate: 1
Stalemate rate: 0
Loss value: 0.013653256930410862, num of party: 20
Win rate: 12
Lose rate: 4
Stalemate rate: 5
Loss value: 4.0531159584134e-07, num of party: 40
Win rate: 32
Lose rate: 4
Stalemate rate: 5
Loss value: 1.2078285180905368e-06, num of party: 60
Win rate: 52
Lose rate: 4
Stalemate rate: 5
Loss value: 1.1920930376163597e-07, num of party: 80
Win rate: 70
Lose rate: 5
Stalemate rate: 6
Loss value: 1.2078285180905368e-06, num of party: 100
Win rate: 89
Lose rate: 5
Stalemate rate: 7
Loss value: 1.1920930376163597e-07, num of party: 120
Win rate: 108
Lose rate: 6
Stalemate rate: 7
Loss value: 1.1920930376163597e-07, num of party: 140
Win rate: 128
Lose rate: 6
Stalemate rate: 7
Loss value: 1.1920930376163597e-07, num of party: 160
Win rate: 147
Lose rate: 7
Stalemate rate: 7
Loss value: 1.1920930376163597e-07, num of party: 180
Win rate: 167
Lose rate: 7
Stalemate rate: 7
Loss value: 1.207

KeyboardInterrupt: 

In [ ]:
for param in policy_net.parameters():
    print(param.grad)